## Processing fold axial traces for geomtry and as orientation constraints
Saves geometry of fold axial traces and creates orientation csv file either side of trace with appropriate dip directions and arbitrary dips  
Only saves orientation data where interpolated contacts are roughly parallel to fold axial traces  


In [ ]:
import geopandas as gpd
import pandas as pd
import numpy as np
import os
import sys
import stat
import functools 
import operator  
import matplotlib
import networkx as nx
import rasterio
from shapely.geometry import Polygon
from map2loop import m2l_utils
from map2loop import m2l_topology
from map2loop import m2l_geometry
from map2loop import m2l_interpolation
from shapely.geometry import shape, Polygon, LineString, Point
from math import sin, cos, atan, atan2, asin, radians, degrees, sqrt, pow, fabs
from geopandas import GeoDataFrame
%matplotlib inline

In [ ]:
test_data_name='test_data3'

test_data_path='../'+test_data_name+'/'

os.chdir(test_data_path)

%run -i "m2l_config.py"
dtm = rasterio.open(dtm_reproj_file)

In [ ]:
local_paths=True
fold_decimate=2        

In [ ]:
fat_step=750         # how much to step out normal to fold axial trace
close_dip=20.0       #dip to assign to all new orientations

#Save fold axial traces 
def save_fold_axial_traces_orientations(path_folds,path_fold_orientations,dataset,c_l,fold_decimate,fat_step,close_dip):
    geology = gpd.read_file(tmp_path+'geol_clip.shp')
    contacts=np.genfromtxt(tmp_path+'interpolation_contacts_scipy_rbf.csv',delimiter=',',dtype='float')
    f=open(output_path+'fold_axial_trace_orientations2.csv','w')
    f.write('X,Y,Z,azimuth,dip,polarity,formation\n')
    folds_clip=gpd.read_file(path_folds,)
    fo=open(path_fold_orientations+'fold_axial_traces.csv',"w")
    fo.write("X,Y,Z,code,type\n")
    dummy=[]
    dummy.append(1)
    for indx,fold in folds_clip.iterrows():
        fold_name=str(fold[c_l['o']])   
        fold_ls=LineString(fold.geometry)
        i=0
        first=True
        for afs in fold_ls.coords:
            if(c_l['fold'] in fold[c_l['f']]):
                if(m2l_utils.mod_safe(i,fold_decimate)==0 or i==int((len(fold_ls.coords)-1)/2) or i==len(fold_ls.coords)-1): #decimate to reduce number of points, but also take mid and end points of a series to keep some shape
                    locations=[(afs[0],afs[1])]                  
                    height=m2l_utils.value_from_raster(dataset,locations)
                    ostr=str(afs[0])+','+str(afs[1])+','+str(height)+','+'FA_'+fold_name+','+fold[c_l['t']].replace(',','')+'\n'
                    fo.write(ostr)  
                    if(not first):
                        l,m=m2l_utils.pts2dircos(lastx,lasty,afs[0],afs[1])
                        midx=lastx+((afs[0]-lastx)/2)
                        midy=lasty+((afs[1]-lasty)/2)
                        midxr=midx+(fat_step*-m)
                        midyr=midy+(fat_step*l)
                        midxl=midx-(fat_step*-m)
                        midyl=midy-(fat_step*l)
                        dip,dipdir=m2l_utils.dircos2ddd(-m,l,cos(radians(close_dip)))
                        if(c_l['syn'] in fold[c_l['t']]):
                            dipdir=dipdir+180
                        mindist=1e9
                        minind=-1
                        for i in range(1,len(contacts)):
                            dist=m2l_utils.ptsdist(contacts[i,0],contacts[i,1],midx,midy)
                            if(dist<mindist):
                                mindist=dist
                                minind=i
                        lc=sin(radians(contacts[minind,2]))
                        mc=cos(radians(contacts[minind,2]))
                        dotprod=fabs((l*lc)+(m*mc))
                        #print(dotprod,midx,midy,minind,contacts[minind,2],l,m,lc,mc)   
                        if(dotprod>0.85):
                            geometry = [Point(midxr,midyr)]
                            gdf = GeoDataFrame(dummy, crs=dst_crs, geometry=geometry)
                            structure_code = gpd.sjoin(gdf, geology, how="left", op="within")
                            if(not str(structure_code.iloc[0]['CODE'])=='nan'):
                                locations=[(midxr,midyr)]                  
                                height=m2l_utils.value_from_raster(dataset,locations)
                                ostr=str(midxr)+','+str(midyr)+','+str(height)+','+str(dipdir)+','+str(int(dip))+',1,'+str(structure_code.iloc[0]['CODE']).replace(" ","_").replace("-","_")+'\n'
                                f.write(ostr)
                            
                            geometry = [Point(midxl,midyl)]
                            gdf = GeoDataFrame(dummy, crs=dst_crs, geometry=geometry)
                            structure_code = gpd.sjoin(gdf, geology, how="left", op="within")
                            if(not str(structure_code.iloc[0]['CODE'])=='nan'):
                                locations=[(midxl,midyl)]                  
                                height=m2l_utils.value_from_raster(dataset,locations)
                                ostr=str(midxl)+','+str(midyl)+','+str(height)+','+str(dipdir+180)+','+str(int(dip))+',1,'+str(structure_code.iloc[0]['CODE']).replace(" ","_").replace("-","_")+'\n'
                                f.write(ostr)
                    first=False
                    lastx=afs[0]
                    lasty=afs[1]
            i=i+1  

    fo.close()
    f.close()
    print("fold axial traces saved as",path_fold_orientations+'fold_axial_traces.csv')
    print("fold axial trace orientations saved as",path_fold_orientations+'fold_axial_trace_orientations.csv')
    
    
save_fold_axial_traces_orientations(tmp_path+'folds_clip.shp',output_path,dtm,c_l,fold_decimate,fat_step,close_dip)